In [1]:
import pickle
import numpy as np

`Read all necessary files that we have save for future using purpose`

- caption_dict -> Caption_description.pkl
- train_caption -> train_description.pkl
- word_to_idx -> Word_to_idx.pkl
- idx_to_word -> Idx_to_word.pkl
- emb_mat -> embedding_matrix.npy

In [5]:
with open('./Caption_description.pkl', 'rb') as f:
    caption_dict = pickle.load(f)
    
with open('./train_description.pkl', 'rb') as f:
    train_caption = pickle.load(f)
    
with open('./Word_to_idx.pkl','rb') as f:
    word_to_idx = pickle.load(f)
    
with open('./Idx_to_word.pkl','rb') as f:
    idx_to_word = pickle.load(f)
    
with open('./embedding_matrix.npy','rb') as f:
    emb_mat = np.load(f)

In [8]:
emb_mat.shape

(1959, 50)

### Model Architecture

In [10]:
from keras.models import Model
from keras.layers import *